In [ ]:
from llm_gan.prompts import llm_generator_prompt, llm_generator_discriminator
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "gpt2"  # Options: "gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
generator_model = GPT2LMHeadModel.from_pretrained(model_name)

model_name = "gpt2"  # Options: "gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
judge_model = GPT2LMHeadModel.from_pretrained(model_name)


# Example usage: Generate text
input_text = llm_generator_prompt("Scary Story", "Horror")
input_ids = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0)
# model(input_ids)

output1 = generator_model.generate(
    input_ids,
    max_length=128,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    temperature=0.7
)

output2 = generator_model.generate(
    input_ids,
    max_length=128,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    temperature=0.7
)

story1 = tokenizer.decode(output1[0][input_ids.shape[1]:], skip_special_tokens=True)
story2 = tokenizer.decode(output2[0][input_ids.shape[1]:], skip_special_tokens=True)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [37]:
from llm_gan.prompts import llm_generator_prompt, llm_generator_discriminator

dis_prompt = llm_generator_discriminator("Scary Story", "Horror", story1, story2)
input_ids = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0)

judge_output = judge_model.generate(
    input_ids,
    max_length=256,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    temperature=0.7
)

judge_text = tokenizer.decode(judge_output[0][input_ids.shape[1]:], skip_special_tokens=True)

judge_text

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' You are a writer.\n\n<ASSERTION> \xa0<title>Scary Stories</title></ASSER>\n\n\n</ASSIGN>'

In [38]:
def extract_answer(text):
    open_tag = "<answer>"
    close_tag = "</answer>"
    open_start = text.rfind(open_tag)
    tag_internal = open_start+len(open_tag)
    close_start = text.find(close_tag, tag_internal)
    if close_start == -1 or open_start == -1: return None
    return text[tag_internal:close_start]


assert extract_answer("<answer>Hello World</answer>") == "Hello World"

assert extract_answer("The <answer>Hello World</answer> <answer>Hello World2</answer>") == "Hello World2"

In [62]:
human_solution = 1

judge_output = extract_answer(judge_text) or 0
judge_reward = 1 - abs(human_solution - judge_output) #probably should set to -1 if got wrong 

judge_input_ids = torch.tensor(tokenizer.encode(judge_text)).unsqueeze(0)
judge_logits = judge_model(judge_input_ids).logits

generator_input_ids = torch.tensor(tokenizer.encode(story1)).unsqueeze(0)
generator_logits = generator_model(generator_input_ids).logits

generator_reward = 1 - judge_reward

generator_loss = -generator_reward* (torch.log(generator_logits.gather(index=generator_input_ids.unsqueeze(-1), dim = -1))).mean()
judge_loss = -judge_reward * (torch.log(judge_logits.gather(index=judge_input_ids.unsqueeze(-1), dim = -1))).mean()

judge_loss


tensor(nan, grad_fn=<MulBackward0>)

In [69]:
torch.log_softmax(generator_logits, dim =-1)

tensor([[[ -6.4946,  -6.6046, -10.0949,  ..., -14.3408, -12.5590,  -6.5812],
         [ -8.3404,  -9.8352, -14.8481,  ..., -17.2496, -14.8426, -12.3221],
         [-10.5373, -10.6072, -13.2085,  ..., -18.2289, -15.6205, -11.0360],
         ...,
         [ -4.2418,  -8.4316, -10.8249,  ..., -16.4601, -16.9106,  -9.5861],
         [ -4.0885,  -6.3354,  -8.1105,  ..., -18.5741, -18.2937,  -7.4168],
         [-15.4605, -14.1716, -12.9280,  ..., -34.5463, -36.6530, -19.2465]]],
       grad_fn=<LogSoftmaxBackward0>)

In [65]:
generator_logits.gather(index=generator_input_ids.unsqueeze(-1), dim = -1)

tensor([[[ -41.5136],
         [-119.5114],
         [ -93.5414],
         [-103.5277],
         [-117.4410],
         [-228.6439],
         [-113.8802],
         [ -70.7703],
         [ -42.0122],
         [ -60.4546],
         [ -60.6491],
         [ -84.5732],
         [ -19.1010],
         [ -89.9902],
         [ -61.2169],
         [   7.5971],
         [ -53.5884],
         [ -85.4939],
         [ -79.1273],
         [ -98.3296],
         [-210.9904]]], grad_fn=<GatherBackward0>)

In [61]:
(judge_logits.gather(index=judge_input_ids.unsqueeze(-1), dim = -1)).mean()

tensor(-73.9761, grad_fn=<MeanBackward0>)

In [50]:
judge_input_ids.size()
judge_logits.size()

torch.Size([1, 32, 50257])

tensor([[[ -41.5136],
         [-119.5114],
         [ -93.5414],
         [-103.5277],
         [-117.4410],
         [-228.6439],
         [-113.8802],
         [ -70.7703],
         [ -42.0122],
         [ -60.4546],
         [ -60.6491],
         [ -84.5732],
         [ -19.1010],
         [ -89.9902],
         [ -61.2169],
         [   7.5971],
         [ -53.5884],
         [ -85.4939],
         [ -79.1273],
         [ -98.3296],
         [-210.9904],
         [ 144.9540],
         [ -45.0791],
         [-218.1480],
         [ -62.3141],
         [ -58.0737],
         [-249.1573],
         [ -87.7768],
         [ -72.2683],
         [  85.4817],
         [   4.3350],
         [  17.5721]]], grad_fn=<GatherBackward0>)